In [1]:
import numpy as np
import pandas as pd

import keras

from keras import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Input

from sklearn.preprocessing import OneHotEncoder

/Users/zfh403/.pyenv/versions/3.6.4/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_df = pd.read_csv("data/train.csv", index_col=0)
target_df = pd.read_csv("data/target.csv", index_col=0)

/Users/zfh403/.pyenv/versions/3.6.4/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
train_df.head()

,user_id,order_id,product_id_tsfm,order_number,days_since_prior_order,size
0,1,431534,1589,5,28.0,1
1,1,431534,4027,5,28.0,1
2,1,431534,4087,5,28.0,1
3,1,431534,4090,5,28.0,1
4,1,431534,4102,5,28.0,1


# Tranform into Sequence

In [4]:
X_train = train_df.groupby("user_id")["product_id_tsfm"].apply(np.hstack)

In [5]:
y_train = target_df.groupby("user_id")["product_id_tsfm"].apply(np.hstack)

In [10]:
# convert to numpy
X_train = X_train.values
y_train = y_train.values

# One Hot Encode Sequence

In [6]:
def one_hot_encode(X_train, n_products):
    return np.eye(n_products)[X_train]

In [11]:
X_train[2]

array([ 1026,  1514,  2439,  3484,  4033, 15401, 17685, 20430, 23970,
        1026,  1058,  3484,  3507,  4032,  1026,  1058,  4032, 15401,
       22344, 26895,  1058,  3484,  4032, 15414, 20430, 26895,  1058,
        2984,  3484,  3507,  4046,  4061,  6106, 26895, 29828,  1058,
        1526,  2984,  3038,  4032,  4061,  6106, 15414,  4061,  6106,
       17734, 29828, 30341,  1026,  1058,  1526,  1562,  3038,  3484,
        3489,  4032,  4061, 23295, 30341,  1058,  3013,  3484,  4032,
       17671, 23295,  1058,  3507,  4032,  4046, 30496])

In [12]:
n_products = len(train_df["product_id_tsfm"].unique())

In [16]:
example = one_hot_encode(X_train[2], n_products)

In [17]:
example.shape

(70, 36055)

In [18]:
len(X_train[2])

70

In [21]:
X_train.shape

(206209,)

# Training Data Generator

In [111]:


g = one_hot_generator()
        
def train_generator(BATCH_SIZE=10):
    x_batch = []
    y_batch = []
    
    i = 0
    for new_x, new_y in g:
        if i > BATCH_SIZE:
            break
        x_batch.append(new_x)
        y_batch.append(new_y)
        i += 1
    yield (np.array(x_batch), np.array(y_batch))

In [72]:
x, y = next(iter(one_hot_generator()))

In [73]:
np.array(x).shape

(1, 50, 36055)

In [49]:
x, y = next(iter(train_generator()))

In [62]:
x.shape

(11,)

In [63]:
x

array([array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]),
       array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]),
       array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]),
       array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]),
    

In [53]:
x.reshape(x.shape).shape

(1, 11)

# Model 

In [127]:
def one_hot_generator():
    for x, y in zip(X_train, y_train):
        x_one_hot = one_hot_encode(x, n_products)
        y_one_hot = one_hot_encode(y, n_products)
        yield (np.array([x_one_hot]), y_one_hot)
        
# TODO: try grouping these into batches (of say 100)

In [164]:
def get_batch():
    i = 0 
    for x, y in zip(X_train, y_train):
        x_one_hot = one_hot_encode(x, n_products)
        y_one_hot = one_hot_encode(y, n_products)
        
        if i == 0:
            X_batch = np.array(x_one_hot)
            y_batch = np.array(y_one_hot)
        elif i % 10 != 0:
            X_batch = np.concatenate((X_batch, x), axis=0)
            y_batch = np.concatenate((y_batch, y), axis=0)
        else:
            yield (X_batch, y_batch)
            i = 0
        
# TODO: try grouping these into batches (of say 100)

In [162]:

for i in range(3):
    X, y = next(iter(one_hot_generator()))
    if i == 0:
        X_batch = np.array(X)
        y_batch = np.array(y)
    else:
        X_batch = np.concatenate((X_batch, X), axis=0)
        y_batch = np.concatenate((y_batch, y), axis=0)

In [165]:
X, y = next(iter(get_batch()))

KeyboardInterrupt: 

In [129]:
model = Sequential()

model.add(LSTM(256, input_shape=(None,n_products)))
model.add(Dropout(0.5))
model.add(Dense(n_products, activation='softmax'))


In [130]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [131]:
model.fit_generator(one_hot_generator(), steps_per_epoch=1000, epochs=5)

Epoch 1/5
  41/1000 [>.............................] - ETA: 1:39:38 - loss: 10.4929 - acc: 0.0000e+00

KeyboardInterrupt: 